In [1]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.geocoding import geocode

In [2]:
Halifax_GIS_Map = GIS(username = "JuanCReyes1", password="Kamituru1.")

places_dict= {"Dartmouth":"Dartmouth,Nova Scotia,Canada",
              "Halifax": "Halifax, Nova Scotia, Canada",
              "Bedford": "Bedford, Nova Scotia, Canada"
              }
Halifax_Covid_Exposures = Halifax_GIS_Map.map(places_dict["Dartmouth"],12)
Halifax_Covid_Exposures.basemap = "streets-night-vector" #map style
Halifax_Covid_Exposures

MapView(layout=Layout(height='400px', width='100%'))

In [3]:
#1 data import
df1 = pd.read_html("http://www.nshealth.ca/covid-exposures")
df2 = pd.read_html("https://www.nshealth.ca/coronavirus-exposures?title=&field_covid_exposure_zone_value=All&page=1")
df_transit = pd.read_html("https://www.nshealth.ca/coronavirus-exposures-transit")
df_transit = df_transit[0].rename(columns={'Route or flight': "Place"})
df3 = pd.concat([df1[0],df2[0],df_transit], ignore_index = True)


#2 data cleaning
##Check For NaN values
if df3.isnull().values.any():
    print("Check for NaN values in our columns")
    print("Place: " + str(df3["Place"].isnull().values.any()))
    print("Potential Exposure Window: " + str(df3["Potential Exposure Window"].isnull().values.any()))
    print("Address: " + str(df3["Address"].isnull().values.any()))
    print("Details: " + str(df3["Details"].isnull().values.any()))
    print("Zone: " + str(df3["Zone"].isnull().values.any()))
    print("Last Updated: " + str(df3["Last Updated"].isnull().values.any()))
    print("Exposure Area: " + str(df3["Exposure area"].isnull().values.any()))
#print("no null values")

#Exposure Area NAN

if df3["Exposure area"].isnull().values.any():
    print("Empty values in exposure area")
    null_exposures_index = df3[df3["Exposure area"].isnull().values].index
    df3["Exposure area"].iloc[null_exposures_index] = df3["Place"].iloc[null_exposures_index]


##Zone Column NAN
if df3['Zone'].isnull().values.any():
    print("Zone value empty due to flights")
    null_zone_index = df3[df3['Zone'].isnull().values].index
    df3.iloc[null_zone_index,4] = "Flight"
    print("Zone Data Cleaning Complete: "+ str(df3["Zone"].isnull().any()))
#print("no null zone values")


#Identify Covid Flights and Transit Routes
print("Attributing Covid Flights to Halifax Stanfield International Airport")
covid_flights = df3[df3["Place"].str.contains("to Halifax")]
covid_bus_routes = df3[(df3["Place"].str.contains("Transit")) | (df3["Place"].str.contains("Route"))]
#Update Covid Flights Address to Halifax Stanfield International Airport:
df3["Address"].iloc[covid_flights.index] = "Halifax Stanfield International Airport"
df3["Zone"].iloc[covid_flights.index] = "Flight"

#Update Covid Routes Address to the Transit Route
df3["Address"].iloc[covid_bus_routes.index] = "Halifax Transit Bridge Terminal"
#df3["Zone"].iloc[covid_bus_routes.index] = "Halifax Transit Bridge Terminal"

#Create a Geolocation Address which will be fed into ArcGIS
print("Creating A Geolocation Address Column For Each Exposure Site...")
df3['Geolocation Address'] = df3[['Place', 'Address']].apply(lambda x: '-'.join(x.map(str)), axis=1)
print("Geolocation Address Column: Included")



print("Creating Geolocation Address Coordinates")
##Create GIS Object!
print("Creating ArcGIS Map Object")
Halifax_GIS_Map = GIS()

df3['Geolocation Address Coords'] = df3['Geolocation Address'].apply(lambda x: geocode(x,out_fields="address,location,City,Country,URL"))
print("Complete")

##Obtaining Latitude and Longitude for each address:
print("Create Latitude & Longitude Columns")
indices = df3['Geolocation Address Coords'].index
longitudeseries = []
latitudeseries = []

print("Creating Latitude and Longitude Columns")
for number in indices:
    #print(number)
    longitudeseries.append(df3['Geolocation Address Coords'][number][0]["location"]["x"])
    latitudeseries.append(df3['Geolocation Address Coords'][number][0]["location"]["y"])

df3["longitude"] = longitudeseries
df3["latitude"] = latitudeseries

##Export to CSV
print("Exporting to CSV")
df3.to_csv('./covidlocations.csv')
print("CSV Complete")

Check for NaN values in our columns
Place: False
Potential Exposure Window: False
Address: True
Details: False
Zone: True
Last Updated: False
Exposure Area: True
Empty values in exposure area
Zone value empty due to flights
Zone Data Cleaning Complete: False
Attributing Covid Flights to Halifax Stanfield International Airport
Creating A Geolocation Address Column For Each Exposure Site...
Geolocation Address Column: Included
Creating Geolocation Address Coordinates
Creating ArcGIS Map Object
Complete
Create Latitude & Longitude Columns
Creating Latitude and Longitude Columns
Exporting to CSV
CSV Complete


In [4]:
geotag_symbol_dict_redbullseye = {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/esriCrimeMarker_86.png","contentType":"image/png","width":16,"height":16}
geotag_symbol_dict_redfirefly = {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"https://static.arcgis.com/images/Symbols/Firefly/FireflyD20.png","contentType":"image/png","width":24,"height":24}
geotag_symbol_dict_radioactive = {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/PeoplePlaces/Radioactive.png","contentType":"image/png","width":16,"height":16}


In [5]:
#plot covid locations
for number in range(0,len(df3['Geolocation Address Coords'])):
#for number in range(0,15):
    location = df3['Geolocation Address Coords'][number][0]["location"]
    popup = {"title": pd.Series(df3["Geolocation Address"])[number], "content": pd.Series(df3["Exposure area"])[number]}
    symbol = geotag_symbol_dict_radioactive
    attributes = {"test": "what"}
    Halifax_Covid_Exposures.draw(shape = location,popup=popup, symbol = symbol ,attributes = "test")
    print("Plotting location number " + str(number) + "/ " +str(len(df3["Geolocation Address Coords"])-1)+": " + pd.Series(df3["Geolocation Address"])[number] )

    
#also plot all healthcare facilities in canada    
covid_canada = Halifax_GIS_Map.content.search("title: Open Database of Healthcare Facilities - Canada")
covid_canada[0]
covid_layer = Halifax_GIS_Map.content.get(covid_canada[0].id)
Halifax_Covid_Exposures.add_layer(covid_layer,options={'opacity':1.2})

Plotting location number 0/ 94: Café Marco Polo-15 Kings Wharf Pl., Dartmouth
Plotting location number 1/ 94: Access Nova Scotia-300 Horseshoe Lake Dr., Halifax
Plotting location number 2/ 94: Pitch & Pivot-240 Brownlow Ave., Dartmouth
Plotting location number 3/ 94: The Old Triangle Irish Alehouse-5136 Prince St, Halifax
Plotting location number 4/ 94: Moxies Bar & Grill-9 Countryview Dr., Dartmouth
Plotting location number 5/ 94: Ela! Greek Taverna-150 Hector Gate, Dartmouth
Plotting location number 6/ 94: Sou’Wester Gift and Restaurant-178 Peggys Point Rd., Peggys Cove
Plotting location number 7/ 94: Hermitage-1460 Lower Water St., Halifax
Plotting location number 8/ 94: Pür & Simple-189 Hector Gate, Dartmouth
Plotting location number 9/ 94: Pitch & Pivot-240 Brownlow Ave., Dartmouth
Plotting location number 10/ 94: Hermitage-1460 Lower Water St., Halifax
Plotting location number 11/ 94: GoodLife Fitness-3711 Joseph Howe Dr., Halifax
Plotting location number 12/ 94: Hermitage-1460 L